datavisualization.R
Jay Sayre - sayrejay (at) Gmail,

Used for data visualization for CAFTA-DR paper

NOTE: cleanuptextable.py MUST BE RUN AFTER RUNNING THIS SCRIPT TO UPDATE GRAPHS IN PAPER

Inputs - 

"municipality_occupation_level_DATASET.csv" -Municipality and occupation level dataset produced by dataassembly.py

"wtoandcaftahstariff.csv" - Merged wto and cafta HS6 tariff data

"tariffhs2.csv" - HS2 level tariff data 

Outputs - 

"hstwodigittrfsTIKZ.tex" - Plots HS2 level tariffs from 1996-2006 as a line graph

"tariffdecrisicTIKZ.tex" - Plots the average tariff at the ISIC two digit level in 2002 against the change in the average ISIC tariff from 2002-2013

"tariffhs6decreaseTIKZ.tex"  - Plots the average tariff at the HS6 level in 2002 against the change in the average HS6 tariff from 2002-2013

In [1]:
library(ggplot2)
library(texreg)
library(tikzDevice)

setwd("~/Dropbox/College/DR_Paper/cafta-dr/Output/")
#setwd("D:/Dropbox/Dropbox (Personal)/College/DR_Paper/")

## INPUTS
munoccdf <- read.csv("municipality_occupation_level_DATASET.csv")
hs6tariff <- read.csv("wtoandcaftahstariff.csv")
hs2tariff <- read.csv("tariffhs2.csv")

## OUTPUTS 
plotdir <- "../Plots/"
hs2timetikz <- paste(plotdir,"hstwodigittrfsTIKZ.tex",sep="")
isictariffchangetikz <- paste(plotdir,"tariffdecrisicTIKZ.tex",sep="")
hs6tariffchangetikz <- paste(plotdir, "tariffhs6decreaseTIKZ.tex",sep="")
hs2tariffchangetikz <- paste(plotdir, "tariffhs2decreaseTIKZ.tex",sep="")

Version:  1.36
Date:     2015-12-08
Author:   Philip Leifeld (Eawag & University of Bern)

Please cite the JSS article in your publications -- see citation("texreg").


In [2]:
### Clean up input data

## CLean up HS6 level tariff data set
hs6tariff$chngtrf <- hs6tariff$duty2013-hs6tariff$X2002AvgRate
hs6tariff$lgchng <- log(-hs6tariff$chngtrf+1)
hs6tariff$lgdty02 <-log(hs6tariff$X2002AvgRate+1)

## Cleanup variables in municipality/occupation level dataset
munoccdf$mean=rowMeans(munoccdf[,c("incforocc13", "firstsourceofincome13",
                                  "totalhhincpercapita13","incpercapita13")],
                       na.rm=TRUE)
munoccdf$chnginc1 <- munoccdf$incforocc13-munoccdf$inc2002
munoccdf$chnginc2 <- munoccdf$firstsourceofincome13-munoccdf$inc2002
munoccdf$chnginc3 <- munoccdf$totalhhincpercapita13-munoccdf$inc2002
munoccdf$chnginc4 <- munoccdf$incpercapita13-munoccdf$inc2002
munoccdf$chnginc5 <- munoccdf$mean-munoccdf$inc2002
munoccdf$chngtrf <- munoccdf$duty13-munoccdf$duty02
munoccdf$lgchng <- log(-munoccdf$chngtrf+1)
munoccdf$lgdty02 <-log(munoccdf$duty02+1)
munoccdf$chngwrk <- munoccdf$numworkers10-munoccdf$numworkers02
munoccdf$occ <- factor(munoccdf$occ)

In [37]:
## Plot tariff level in 2002 compared to tariff change
bf <- lm(munoccdf$chngtrf~munoccdf$duty02)
rsq <- summary(bf)$r.squared
rsq <- signif(rsq,4)
beta1 <- coef(summary(bf))[2,"Estimate"]
beta1 <- signif(beta1,4)
se1 <- coef(summary(bf))[2,"Std. Error"]
se1 <- signif(se1,4)
numobs <- summary(bf)$df[2]
line <- paste("Coefficient Estimate: ", beta1,
              #", se: ",se1, # Need to subset this down to 15 unique occs
              ", R^2: ",rsq,", N=15",sep="")
              #numobs,

p1 <- ggplot(munoccdf, aes(x=duty02, y=chngtrf)) +
  coord_equal() + geom_jitter(aes(colour=occ),size=.75) + #geom_point(shape=1) +
  #geom_text(aes(label=occ)) + # Could do this perhaps
  ggtitle("Tariff Changes 2002-2013 based on Pre-CAFTA Tariff Levels
  \n(ISIC 2 digit Import Competing Occupation)") + geom_smooth(method=lm)+ 
  xlab("2002 pre-CAFTA duty") + ylab("Decrease in duty rate") +
  theme_classic()
tikz(file=isictariffchangetikz,standAlone=TRUE,sanitize=TRUE)
plot.new()
print(p1)
mtext(line,line=-35)
dev.off()

pdf 
  2

In [38]:
## Do same as above but at HS6 level
bf <- lm(hs6tariff$chngtrf~hs6tariff$X2002AvgRate)
rsq <- summary(bf)$r.squared
rsq <- signif(rsq,4)
beta1 <- coef(summary(bf))[2,"Estimate"]
beta1 <- signif(beta1,4)
se1 <- coef(summary(bf))[2,"Std. Error"]
se1 <- signif(se1,4)
numobs <- summary(bf)$df[2]
line <- paste("Coefficient Estimate: ", beta1,
              ", se: ",se1,", R^2: ",rsq,", N=",numobs,sep="")

p2 <- ggplot(hs6tariff, aes(x=X2002AvgRate, y=chngtrf))+
  coord_equal() + geom_point(shape=1) +
  ggtitle("Tariff Changes 2002-2013 based on Pre-CAFTA Tariff Levels\n(HS6 Code)") +
  geom_smooth(method=lm)+
  xlab("2002 pre-CAFTA duty") + ylab("Decrease in duty rate") +
  theme_classic()
tikz(file=hs6tariffchangetikz,standAlone=TRUE,sanitize=TRUE) 
plot.new()
print(p2)
mtext(line,line=-36)
dev.off()

Warning message:
: Removed 30 rows containing non-finite values (stat_smooth).Warning message:
: Removed 30 rows containing missing values (geom_point).

pdf 
  2

In [5]:
## Plot HS2 average tariffs from 1996-2006
hs2tariff$HS2 <- factor(hs2tariff$HS2)

p4 <- ggplot(data=hs2tariff, aes(x=year, y=duty, group=HS2)) +
    geom_line(aes(colour=HS2)) +
    geom_point(aes(colour=HS2)) +
    xlab("Year") + ylab("Tariff Rate") +
    theme_classic() + ylim(0,20) +
    theme(legend.position = "none") +
    ggtitle("Tariff Changes pre-CAFTA (HS 2 digit level)")
tikz(file=hs2timetikz,standAlone=TRUE,sanitize=TRUE) 
print(p4)
dev.off()

Warning message:
: Removed 18 rows containing missing values (geom_path).Warning message:
: Removed 18 rows containing missing values (geom_point).

pdf 
  2

In [6]:
p5 <- ggplot(munoccdf, aes(x=chngtrf, y=chnginc3)) +
  #coord_equal() + 
  #geom_jitter(aes(colour=occ),size=.75) + 
  geom_point(shape=1) +
  #geom_text(aes(label=occ)) + # Could do this perhaps
  ggtitle("Hi") + geom_smooth(method=lm)+ 
  xlab("x") + ylab("y") +
  theme_classic()
#p4